<a href="https://colab.research.google.com/github/mgkid3310/mnk-dqn/blob/feature%2Ffix/mnk_game_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm

# mnk 환경 정의

In [2]:
class Environment():
    
    def __init__(self):
        """
        보드는 0으로 초기화된 25개의 배열로 준비
        게임종료 : done = True
        """
        self.board_a = np.zeros(25)
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = False

    def move(self, p1, p2, player):
        """
        # 각 플레이어가 선택한 행동을 표시 하고 게임 상태(진행 또는 종료)를 판단
        # p1 = 1, p2 = -1로 정의
        # 각 플레이어는 행동을 선택하는 select_action 메서드를 가짐
        """
        if player == 1:
            pos = p1.select_action(env)
        else:
            pos = p2.select_action(env)
        
        self.board_a[pos] = player # 보드에 플레이어의 선택을 표시
        if self.print:
            print(player)
            self.print_board()
        self.end_check(player) # 게임이 종료상태인지 아닌지를 판단
        
        return  self.reward, self.done

    def get_action(self):
        """
        현재 보드 상태에서 가능한 행동(둘 수 있는 장소)을 탐색하고 리스트로 반환
        """
        observation = []
        for i in range(25):
            if self.board_a[i] == 0:
                observation.append(i)
        return observation
    
    def end_check(self,player):
        """
        게임이 종료(승패 또는 비김)됐는지 판단
        승패 조건은 가로, 세로, 대각선 이 -1 이나 1 로 동일할 때 
        0 1 2 3 4
        5 6 7 8 9
        10 11 12 13 14
        15 16 17 18 19
        20 21 22 23 24
        """
        end_condition = (
                            (0,1,2,3),(1,2,3,4),(5,6,7,8),(6,7,8,9),(10,11,12,13),(11,12,13,14),(15,16,17,18),(16,17,18,19),(20,21,22,23),(21,22,23,24), # 가로
                            (0,6,12,17),(6,12,17,22),(1,7,13,19),(5,11,17,23), # 왼쪽 위 -> 오른쪽 아래 대각선
                            (4,8,12,16),(8,12,16,20),(3,7,11,15),(9,13,17,21), # 오른쪽 위 -> 왼쪽 아래 대각선
                            (0,5,10,15),(5,10,15,20),(1,6,11,16),(6,11,16,21),(2,7,12,17),(7,12,17,22),(3,8,13,18),(8,13,18,23),(4,9,14,19),(9,14,19,24) # 세로
                        )
        for line in end_condition:
            if self.board_a[line[0]] == self.board_a[line[1]] \
                and self.board_a[line[1]] == self.board_a[line[2]] \
                and self.board_a[line[2]] == self.board_a[line[3]] \
                and self.board_a[line[0]] != 0:
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = player
                return
        # 비긴 상태는 더는 보드에 빈 공간이 없을때
        observation = self.get_action()
        if (len(observation)) == 0:
            self.done = True
            self.reward = 0            
        return
        
    def print_board(self):
        """
        현재 보드의 상태를 표시 
        p1 = O, p2 = X    
        """
        print("+----+----+----+----+----+")
        for i in range(5):
            for j in range(5):
                if self.board_a[5*i+j] == 1:
                    print("|  O",end=" ")
                elif self.board_a[5*i+j] == -1:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+----+----+")

# 사람

In [3]:
class Human_player():
    def __init__(self):
        self.name = "Human player"
        
    def select_action(self, env):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = env.get_action()
            print("possible actions = {}".format(available_action))

            # 상태 번호 표시
            print("+----+----+----+----+----+")
            print("+  0  +  1  +  2  +  3  +  4  +")
            print("+----+----+----+----+----+")
            print("+  5  +  6  +  7  +  8  +  9  +")
            print("+----+----+----+----+----+")
            print("+ 10  + 11  + 12  + 13  + 14  +")
            print("+----+----+----+----+----+")
            print("+ 15  + 16  + 17  + 18  + 19  +")
            print("+----+----+----+----+----+")
            print("+ 20  + 21  + 22  + 23  + 24  +")
            print("+----+----+----+----+----+")
                        
            action = int(input("Select action(human) : ")) # 키보드로 가능한 행동을 입력 받음
            
            if action in available_action: return action # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            else: print("You selected wrong action") # 아니면 행동 입력을 반복

# Q learning

In [4]:
class Q_learning_player():
    
    def __init__(self):
        self.name = "Q_player"
        self.qtable = {} # Q-table을 딕셔너리로 정의
        self.epsilon = 1 # e-greedy 계수
        self.learning_rate = 0.1 # 학습률
        self.gamma=0.9
        self.print = False

    def select_action(self, env):
        """
        policy에 따라 상태에 맞는 행동을 선택
        """
        action = self.policy(env)
        if self.print: print("{} : select action".format(action))
        return action 
        
    def policy(self, env):
        if self.print:
            print("-----------   policy start -------------")
        available_action = env.get_action() # 행동 가능한 상태를 저장
        qvalues = np.zeros(len(available_action)) # 행동 가능한 상태의 Q-value를 저장

        if self.print: print("{} : available_action".format(available_action))

        # 행동 가능한 상태의 Q-value를 조사
        for i, act in enumerate(available_action):
            key = (tuple(env.board_a),act)
            # 현재 상태를 경험한 적이 없다면(딕셔너리에 없다면) 딕셔너리에 추가(Q-value = 0)
            if self.qtable.get(key) ==  None:                
                self.qtable[key] = 0
            # 행동 가능한 상태의 Q-value 저장
            qvalues[i] = self.qtable.get(key)

        # e-greedy
        # 가능한 행동들 중에서 Q-value가 가장 큰 행동을 저장
        greedy_action = np.argmax(qvalues)                    

        pr = np.zeros(len(available_action))
        if self.print:
            print("{} : self.epsilon = {}".format(self.epsilon))
            print("{} : greedy_action".format(greedy_action))
            print("{} : qvalues".format(greedy_action, qvalues[greedy_action]))

        # max Q-value와 같은 값이 여러개 있는지 확인한 후 double_check에 상태를 저장
        double_check = (np.where(qvalues == np.max(qvalues),1,0))

        #  여러개 있다면 중복된 상태중에서 다시 무작위로 선택    
        if np.sum(double_check) > 1:
            if self.print:
                print("{} : double_check".format(np.round(double_check,2)))
            double_check = double_check/np.sum(double_check)
            greedy_action =  np.random.choice(range(0,len(double_check)), p=double_check)
            if self.print:
                print("{} : greedy_action".format(greedy_action))
                print("{} : double_check".format(np.round(double_check,2)))
                print("{} : selected state".format(available_state[greedy_action]))
        # e-greedy로 행동들의 선택 확률을 계산
        pr = np.zeros(len(available_action))

        for i in range(len(available_action)):
            if i == greedy_action:
                pr[i] = 1 - self.epsilon + self.epsilon/len(available_action)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i],2)))
            else:
                pr[i] = self.epsilon / len(available_action)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i],2)))
        action = np.random.choice(range(0,len(available_action)), p=pr)

        if self.print:
            print("pr = {}".format(np.round(pr,2)))
            print("action = {}".format(action))
            print("state[action] = {}".format(state[action]))
            print("-----------   policy end -------------")

        return available_action[action]    
    
    def learn_qtable(self,board_bakup, action_backup, env, reward):
        key = (board_bakup,action_backup) # 현재 상태와 행동을 키로 저장
        if self.print:
            print("-----------   learn_qtable start -------------")
            print("{} : board_bakup, {} : action_backup, {} : reward".format(board_bakup, action_backup, reward))
            print("{} : key".format(key))

        if env.done == True: # 게임이 끝났을 경우 학습
            if self.print:
                print("{} : before self.qtable[key]".format(self.qtable[key]))

            self.qtable[key] += self.learning_rate*(reward - self.qtable[key])
            
            if self.print:
                print("{} : after self.qtable[key]".format(self.qtable[key]))
        else: # 게임이 진행중일 경우 학습
            available_action = env.get_action()        
            qvalues = np.zeros(len(available_action))

            for i, act in enumerate(available_action):
                next_key = (tuple(env.board_a),act)
                # 다음 상태를 경험한 적이 없다면(딕셔너리에 없다면) 딕셔너리에 추가(Q-value = 0)
                if self.qtable.get(next_key) ==  None:                
                    self.qtable[next_key] = 0
                qvalues[i] = self.qtable.get(next_key)

            # maxQ 조사
            maxQ = np.max(qvalues)  
            
            if self.print:
                print("{} : before self.qtable[key]".format(self.qtable[key]))
            
            self.qtable[key] += self.learning_rate*(reward + self.gamma * maxQ - self.qtable[key])
            
            if self.print:
                print("{} : after self.qtable[key]".format(self.qtable[key]))
                
        if self.print:
            print("-----------   learn_qtable end -------------")


# Q learning 훈련

In [6]:
p1_Qplayer = Q_learning_player()
p2_Qplayer = Q_learning_player()

# 입실론은 0.5로 설정
p1_Qplayer.epsilon = 0.5
p2_Qplayer.epsilon = 0.5

p1_score = 0
p2_score = 0
draw_score = 0

max_learn = 1000

for j in tqdm(range(max_learn)):
    np.random.seed(j)
    env = Environment()
    
    for i in range(10000):
        # p1 행동 선택
        player = 1
        pos = p1_Qplayer.policy(env)
        p1_board_backup = tuple(env.board_a)
        p1_action_backup = pos
        env.board_a[pos] = player
        env.end_check(player)
        
        # 게임이 종료상태라면 각 플레이어의 Q-table을 학습
        if env.done == True:
            if env.reward == 0: # 비겼으면 보수 0
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, 0)
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, 0)
                draw_score += 1
                break
            else:
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, 1) # p1이 이겼으므로 보상 +1
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, -1) # p2이 졌으므로 보상 -1
                p1_score += 1
                break
            
        # 게임이 끌나지 않았다면 p2의 Q-talble을 학습 (게임 시작직후에는 p2 는 학습할수 없음)
        if i != 0:
            p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, -0.01)
        
        # p2 행동 선택
        player = -1
        pos = p2_Qplayer.policy(env)
        p2_board_backup = tuple(env.board_a)
        p2_action_backup = pos
        env.board_a[pos] = player
        env.end_check(player)
        
        if env.done == True:
            if env.reward == 0: # 비겼으면 보수 0
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, 0)
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, 0)
                draw_score += 1
                break
            
            else:
                p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, -1) # p1이 졌으므로 보상 -1
                p2_Qplayer.learn_qtable(p2_board_backup, p2_action_backup, env, 1) # p2이 이겼으므로 보상 +1
                p2_score += 1
                break
    
        # 게임이 끌나지 않았다면 p1의  Q-talble 학습
        p1_Qplayer.learn_qtable(p1_board_backup, p1_action_backup, env, -0.01)    
    
print("p1 = {} p2 = {} draw = {}".format(p1_score,p2_score,draw_score))
print("end train")

100%|██████████| 1000/1000 [00:06<00:00, 165.22it/s]

p1 = 560 p2 = 388 draw = 52
end train


# DQN

In [7]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Flatten, Conv2D


class DQN_player():
    
    def __init__(self):
        self.name = "DQN_player"
        self.epsilon = 1
        self.learning_rate = 0.1
        self.gamma=0.9
        
        self.main_network = self.make_network()
        self.target_network = self.make_network()
        self.copy_network() # 메인 신경망의 가중치를 타깃 신경망의 가중치로 복사
        
        self.print = False
        self.print1 = False
        self.count = np.zeros(25)
        self.win = np.zeros(25)
        self.begin = 0
        self.e_trend = []
        
    def make_network(self):
        """
        신경망 생성
        """
        self.model = Sequential()
        self.model.add(Conv2D(16, (5, 5), padding='same', activation='relu', input_shape=(5,5,4)))
        self.model.add(Conv2D(32, (5, 5), padding='same', activation='relu'))
        self.model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
        self.model.add(Flatten())
        self.model.add(Dense(256, activation='tanh'))
        self.model.add(Dense(128, activation='tanh'))
        self.model.add(Dense(64, activation='tanh'))
        self.model.add(Dense(25))
        print(self.model.summary())

        self.model.compile(optimizer = SGD(lr=0.01), loss = 'mean_squared_error', metrics=['mse'])
        
        return self.model
    
    def copy_network(self):
        """
        신경망 복사
        """
        self.target_network.set_weights(self.main_network.get_weights())
        
    def save_network(self, name):
        filename = name + '_main_network.h5'
        self.main_network.save(filename)
        print("end save model")

        
    def state_convert(self, board_a):
        """
        1차원 배열의 보드상태를 2차원으로 변환
        """
        d_state = np.full((5,5,4),0.1)
        for i in range(25):
            if board_a[i] == 1:
                d_state[i//5,i%5,0] = 1
            elif board_a[i] == -1:
                d_state[i//5,i%5,1] = 1
            else:
                pass
        return d_state
    
    
    def select_action(self, env):
        action = self.policy(env)

        if self.print1:
            print("{} : policy state".format(available_state))
            print("{} : qvalues".format(np.round(qvalues,5)))
            print("{} : select action".format(action))
            
        return action 
        
    def policy(self, env):
        if self.print:
            print("-----------   policy start -------------")
        
        available_state = env.get_action() # 행동 가능한 상태를 저장
        
        state_2d = self.state_convert(env.board_a)
        x = np.array([state_2d],dtype=np.float32).astype(np.float32)
        qvalues = self.main_network.predict(x)[0,:]
        
        if self.print:
            print("{} : policy state".format(available_state))
            print("{} : qvalues".format(np.round(qvalues,5)))
        
        available_state_qvalues = qvalues[available_state] # 행동 가능한 상태의 Q-value를 저장

        if self.print:
            print("{} : available_state_qvalues".format(np.round(available_state_qvalues,5)))
        
        greedy_action = np.argmax(available_state_qvalues) # max Q-value를 탐색한 후 저장
        if self.print:
            print("{} : self.epsilon".format(self.epsilon))
            print("{} : greedy_action".format(greedy_action))
            print("{} : qvalue = {}".format(available_state_qvalues[greedy_action]))
        
        # max Q-value와 같은 값이 여러개 있는지 확인한 후 double_check에 상태를 저장
        double_check = (np.where(qvalues == np.max(available_state[greedy_action]),1,0))
        
        #  여러개 있다면 중복된 상태중에서 다시 무작위로 선택    
        if np.sum(double_check) > 1:
            if self.print:
                print("{} : double_check".format(np.round(double_check,4)))
            double_check = double_check/np.sum(double_check)
            greedy_action =  np.random.choice(range(0,len(double_check)), p=double_check)
            if self.print:
                print("{} : greedy_action".format(greedy_action))
                print("{} : double_check".format(np.round(double_check,4)))
                print("{} : selected state".format(available_state[greedy_action]))
        
        # ε-greedy
        pr = np.zeros(len(available_state))

        for i in range(len(available_state)):
            if i == greedy_action:
                pr[i] = 1 - self.epsilon + self.epsilon/len(available_state)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i],4)))
            else:
                pr[i] = self.epsilon / len(available_state)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i],4)))

        action = np.random.choice(range(0,len(available_state)), p=pr)        
        
        if self.print:
            print("{} : pr".format(np.round(pr,4)))
            print("{} : action".format(action))
            print("{} : state[action]".format(available_state[action]))
            print("-----------   policy end -------------")

        if len(available_state) == 25:
            self.count[action] +=1
            self.begin = action
            
        return available_state[action]        
        
    def learn_dqn(self,board_bakup, action_backup, env, reward):
        
        # 입력을 2차원으로 변환한 후, 메인 신경망으로 q-value를 계산
        new_state = self.state_convert(board_bakup)
        x = np.array([new_state],dtype=np.float32).astype(np.float32)
        qvalues = self.main_network.predict(x)[0,:]
        before_action_value = copy.deepcopy(qvalues)
        delta = 0
        
        if self.print:
            print("-----------   learn_qtable start -------------")
            print("{} : board_bakup".format(board_bakup))
            print("{} : action_backup".format(action_backup))
            print("{} : reward = {}".format(reward))
            
        if env.done == True:
            if reward == 1:
                self.win[self.begin] += 1
            if self.print:
                print("{} : delta".format(delta))
                print("{} : before update : actions[action_backup]".format(np.round(qvalues[action_backup],5)))
                print("1  : new_qvalue")
            
            # 게임이 좀료됐을때 신경망의 학습을 위한 정답 데이터를 생성
            qvalues[action_backup] = reward
            y=np.array([qvalues],dtype=np.float32).astype(np.float32)
            # 생성된 정답 데이터로 메인 신경망을 학습
            self.main_network.fit(x, y, epochs=10, verbose=0)
            
            if self.print:
                after_action_value = copy.deepcopy(self.main_network.predict(x)[0,:])
                delta = after_action_value - before_action_value
                print("{} : before_action_value id = {}".format(np.round(before_action_value,5),id(before_action_value)))
                print("{} : target_action_value id = {}".format(np.round(target_action_value,5),id(target_action_value)))
                print("{} : after_action_value id = {}".format(np.round(after_action_value,5),id(after_action_value)))
                print("{} : delta action value".format(np.round(delta,5)))
                state = ((0 for _ in range(15)))
                state_2d = self.state_convert(state)
                x = np.array([state_2d],dtype=np.float32).astype(np.float32)
                qvalues = self.main_network.predict(x)[0,:]
                print("{} : initial state qvalues".format(np.round(qvalues,5)))

        else:
            # 게임이 진행중일때  신경망의 학습을 위한 정답 데이터를 생성
            # 현재 상태에서 최고 Q 값을 계산
            new_state = self.state_convert(env.board_a)
            next_x = np.array([new_state],dtype=np.float32).astype(np.float32)
            next_qvalues = self.target_network.predict(next_x)[0,:]
            available_state = env.get_action()
            maxQ = np.max(next_qvalues[available_state])            
            
            if self.print:
                print("{} : old_qvalue".format(np.round(before_action_value[action_backup],5)))
                print("{} : next_qvalue".format(np.round(next_qvalues,5)))
                print("{} : available_state".format(np.round(available_state,5)))
                print("{} : maxQ".format(np.round(maxQ,5)))
            
            delta = self.learning_rate*(reward + self.gamma * maxQ - qvalues[action_backup])
            
            if self.print:
                print("{} : delta".format(np.round(delta,5)))
                print("{} : before_update_qvalues".format(np.round(qvalues,5)))
                print("{} : before_update_qvalue".format(np.round(qvalues[action_backup],5)))
                
            qvalues[action_backup] += delta
            
            if self.print:
                print("{} : after_update_qvalue".format(np.round(qvalues[action_backup],5)))            
                print("{} : before_update_qvalues".format(np.round(qvalues,5)))
                target_action_value = copy.deep(qvalues)
                print("{} : new_qvalues".format(np.round(qvalues,5)))            
                print("{} : target_action_value id = {}".format(np.round(target_action_value,5),target_action_value))            
            # 생성된 정답 데이터로 메인 신경망을 학습
            y=np.array([qvalues],dtype=np.float32).astype(np.float32)
            self.main_network.fit(x, y, epochs = 10, verbose=0)
            
            if self.print:
                after_action_value = copy.deepcopy(self.main_network.predict(x)[0,:])
                delta = after_action_value - before_action_value
                print("{} : before_action_value id = {}".format(np.round(before_action_value,5),id(before_action_value)))
                print("{} : target_action_value id = {}".format(np.round(target_action_value,5),id(target_action_value)))
                print("{} : after_action_value id = {}".format(np.round(after_action_value,5),id(after_action_value)))
                print("{} : delta action value".format(np.round(delta,5)))
                state = ((0 for _ in range(25)))
                state_2d = self.state_convert(state)
                x = np.array([state_2d],dtype=np.float32).astype(np.float32)
                qvalues = self.main_network.predict(x)[0,:]
                print("{} : initial state qvalues".format(np.round(qvalues,5)))

            
        if self.print:
            print("-----------   learn_qtable end -------------")

# DQN 훈련

In [9]:
np.random.seed(0)

p1_DQN = DQN_player()
print_opt = False
p1_DQN.print = print_opt
p1_DQN.print1 = print_opt

p1_score = 0
p2_score = 0
draw_score = 0

max_learn = 1000

trend = []

p2 = p1_Qplayer
p2.epsilon = 0.5
p2.print = False
p2.print1 = False
        
print("p2 player is {}".format(p2.name))

for j in tqdm(range(max_learn)):
    np.random.seed(j)
    env = Environment()
    
    # 시작할 때 메인 신경망의 가중치를 타깃 신경망의 가중치로 복사
    p1_DQN.epsilon = 0.7
    p1_DQN.copy_network()

    for i in range(10000):
        # p1 행동을 선택
        player = 1
        pos = p1_DQN.policy(env)

        p1_board_backup = tuple(env.board_a)
        p1_action_backup = pos

        env.board_a[pos] = player
        env.end_check(player)

        # 게임 종료라면
        if env.done == True:
            # p1의 승리이므로 마지막 행동에 보상 +1
            # p2는 마지막 행동에 보상 -1
            # p1 행동의 결과는 이기거나 비기거나
            if env.reward == 0:
                p1_DQN.learn_dqn(p1_board_backup, p1_action_backup, env, 0)
                draw_score += 1
                break
            else:
                p1_DQN.learn_dqn(p1_board_backup, p1_action_backup, env, 1)
                p1_score += 1
                break

        # p2 행동을 선택
        player = -1
        pos = p2.select_action(env)
        env.board_a[pos] = player
        env.end_check(player)

        if env.done == True:
            # p2승리 = p1 패배 마지막 행동에 보상 -1
            # 비기면 보상 : 0
            if env.reward == 0:
                p1_DQN.learn_dqn(p1_board_backup, p1_action_backup, env, 0)
                draw_score += 1
                break
            else:
                # 지면 보상 : -1
                p1_DQN.learn_dqn(p1_board_backup, p1_action_backup, env, -1)
                p2_score += 1
                break
        # 게임이 끝나지 않았다면 p1의 Q-talble 학습
        p1_DQN.learn_dqn(p1_board_backup, p1_action_backup, env, 0)

    # 5게임마다 메인 신경망의 가중치를 타깃 신경망의 가중치로 복사
    if j%5 == 0:
        p1_DQN.copy_network()

print("p1 = {} p2 = {} draw = {}".format(p1_score,p2_score,draw_score))
print("end learn")

p1_DQN.save_network("p1_DQN")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 5, 5, 16)          1616      
                                                                 
 conv2d_7 (Conv2D)           (None, 5, 5, 32)          12832     
                                                                 
 conv2d_8 (Conv2D)           (None, 5, 5, 64)          51264     
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_8 (Dense)             (None, 256)               409856    
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dense_10 (Dense)            (None, 64)               

  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 14ms/step


  0%|          | 1/1000 [00:04<1:16:40,  4.61s/it]

1/1 [==============================] - 0s 15ms/step


...

1/1 [==============================] - 0s 19ms/step


100%|█████████▉| 999/1000 [41:25<00:03,  3.21s/it]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 1000/1000 [41:28<00:00,  2.49s/it]

p1 = 614 p2 = 325 draw = 61
end learn
end save model


# 게임 진행

In [10]:
np.random.seed(0)

# p1 = Human_player()
# p2 = Human_player()

# p1 = p1_Qplayer
# p1.epsilon = 0
p2 = p2_Qplayer
p2.epsilon = 0

p1 = p1_DQN
p1.epsilon = 0

# 지정된 게임 수를 자동으로 두게 할 것인지 한게임씩 두게 할 것인지 결정
auto = True # : 지정된 판수(games)를 자동으로 진행 
# auto = False # : 한판씩 진행
# auto 모드의 게임수
games = 100

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0

if auto: # 자동 모드 실행
    for j in tqdm(range(games)):
        np.random.seed(j)
        env = Environment()
        for i in range(10000):
            # p1 과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1,p2,(-1)**i)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    p1_score += 1
                elif reward == -1:
                    p2_score += 1
                else:
                    draw_score += 1
                break

else: # 한 게임씩 진행하는 수동 모드
    np.random.seed(1)
    while True:
        env = Environment()
        env.print = False
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            env.print_board()
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        # 최종 결과 출력        
        print("final result")
        env.print_board()

        # 한게임 더? 최종 결과 출력 
        answer = input("More Game? (y/n)")
        if answer == 'n':
            break           

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))
                

pl player : DQN_player
p2 player : Q_player


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 [==============================] - 0s 15ms/step


  1%|          | 1/100 [00:00<00:32,  3.09it/s]

1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 16ms/step


 99%|█████████▉| 99/100 [00:37<00:00,  2.49it/s]

1/1 [==============================] - 0s 16ms/step


100%|██████████| 100/100 [00:37<00:00,  2.63it/s]

p1(DQN_player) = 88 p2(Q_player) = 11 draw = 1
